In [1]:
class Vertex:
    def __init__(self, performer):
        self.name = performer
        self.adjacent = []
        self.edge = []

    def __str__(self):
        res = ""
        res += self.name
        zipped = zip(self.edge, self.adjacent)
        for movie, costar in zipped:
            res += "|{}/{}".format(movie, costar)
        return res

In [2]:
v = Vertex("Brad Pitt")
v.adjacent = ["Ed Norton", "Julia Roberts", "Anthony Hopkins"]
v.edge = ["Fight Club", "Full Frontal", "Meet Joe Black"]

In [3]:
print(v)

Brad Pitt|Fight Club/Ed Norton|Full Frontal/Julia Roberts|Meet Joe Black/Anthony Hopkins


In [5]:
myfile = open('../data/data_a6/movies.txt','r',encoding="ISO-8859-1")
document = myfile.read()
data = document.split('\n')

# testing
#print(data[0])
#print(data[1])

In [6]:
def build_graph(data):
    graph = {}
    for listing in data:
        listing_elements = listing.split('/')
        movie = listing_elements[0] # first listing element is always the movie, not a performer
        performers = listing_elements[1:] # the other listing elements are performers
        for i, performer in enumerate(performers):
            if performer not in graph:
                graph[performer] = Vertex(performer)
            for j, costar in enumerate(performers):
                if i == j: # prevents linking a performer to themselves
                    continue
                graph[performer].adjacent.append(costar)
                graph[performer].edge.append(movie)
    return graph

In [7]:
graph = build_graph(data)

In [8]:
# Breadth-first-search using the Vertex class.

from collections import deque

# Run bfs and find the path from the Vertex start to the Vertex goal.
# Return that path as a list of Vertex objects.  If there is no path, return None.
def bfs(start, goal):
    frontier = deque()      # the queue of vertices we're going to visit next
    frontier.append(start)  # put the start vertex in our list of those to visit
    
    backpointer = {}
    backpointer[start] = None   # the start vertex has no backpointer
    backpointer_movie = {}
    backpointer_movie[start] = None
        
    # Keep going while there's at least one visited vertex that we have not
    # explored from yet.
    while len(frontier) > 0: 
        vertex = frontier.popleft()
        
        if vertex == goal: # If we're done, retrace the path from the goal to the start.
            path = []        
            while vertex != None: # only our start should have a None backpointer along our path
                if( backpointer_movie[vertex] != None ):
                    path.append(vertex.name + " (" + backpointer_movie[vertex] + ")")
                else:
                    path.append(vertex.name)
               
                vertex = backpointer[vertex]
                #print( vertex.name, backpointer_movie[vertex] )
            return path
        
        else: # keep exploring
            i = 0;
            for neighbor in vertex.adjacent:
                movie = vertex.edge[i]
                # Visit the vertex only if it hasn't been visited yet.  It has been
                # visited if and only if it has a backpointer.
                if not graph[neighbor] in backpointer:
                    # Set up the backpointer.  This will only happen once for each vertex,
                    # since a vertex is put in the frontier and visited only once.
                    backpointer[graph[neighbor]] = vertex
                    backpointer_movie[graph[neighbor]] = movie
                    frontier.append(graph[neighbor])
                i = i + 1
                    
    # If we get here, we've run out of vertices to explore before reaching the goal.
    return None

In [9]:
start = "Hanks, Tom"
goal = "Bacon, Kevin"
if( (start not in graph) or (goal not in graph) ):
    print( "no path found")
else:
    path = bfs(graph[start],graph[goal])
    path = path[::-1] # reverse (start --> goal)
    print(path)

['Hanks, Tom', 'Bacon, Kevin (Apollo 13 (1995))']
